## Lecture du réseau routier

In [1]:
import geopandas as gpd
import momepy
import networkx as nx

file_reseau = "C:/Users/David/Optim/Optimisation/donnees/Troncon route Pacy et environ.shx"

# Lire le shapefile avec GeoPandas
gdf = gpd.read_file(file_reseau)
# Vérifier que la géométrie contient des LineStrings
if not all(gdf.geometry.type.isin(["LineString", "MultiLineString"])):
    raise ValueError("Le fichier doit contenir uniquement des lignes.")


# Convertir en graphe avec la méthode 'primal' (chaque intersection est un nœud)
g_reseau = momepy.gdf_to_nx(gdf, approach='primal')


c:\Users\David\Optim\Optimisation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lecture fichier points de livraision

In [2]:
file_livr = "C:/Users/David/Optim/Optimisation/donnees/Points livraison.shx"

# Lire le shapefile avec GeoPandas
points_livr = gpd.read_file(file_livr)

## Recherche des points du réseau les plus près des points de livraison

In [3]:
nodes_livr = points_livr.geometry.get_coordinates()
nodes_livr['type'] = points_livr['type']

In [7]:
nodes_livr

,x,y,type
0,583009.585282,6.880014e+06,depart
1,581803.833593,6.880498e+06,c1
2,581601.060453,6.880422e+06,c2
3,581520.767733,6.880556e+06,c3
4,581383.317484,6.880660e+06,c5
5,581331.603529,6.880862e+06,c5
6,581247.228129,6.881116e+06,c7
7,581584.729730,6.880942e+06,c9
8,581777.976615,6.880784e+06,c10
9,582001.163158,6.880950e+06,c12


In [19]:
def type_point_reseau(g_reseau,nodes_livr):

    import numpy as np
    nodes_list = list(g_reseau.nodes)
    dico_attribut_nodes = dict()
    capitals = list()
    coordinates = dict()

    for idx, row in nodes_livr.iterrows():
        x_liv = row['x']
        y_liv = row['y']
                
        nearest_node = nodes_list[0]
        X = nearest_node[0]
        Y = nearest_node[1]
        capitals.append(row['type'])


        d_nearest = np.sqrt((x_liv-X)**2 + (y_liv-Y)**2)

        for n in nodes_list[1:]:
            X = n[0]
            Y = n[1]

            d = np.sqrt((x_liv-X)**2 + (y_liv-Y)**2)
            if d < d_nearest:
                nearest_node = n
                d_nearest = d

        coordinates[row['type']] = nearest_node
        
        dico_attribut_nodes[nearest_node] = {'type':row['type']}

    for n in nodes_list:
        if n not in dico_attribut_nodes.keys():
            dico_attribut_nodes[n] = {'type':'interieur'}   

        
    return dico_attribut_nodes,capitals, coordinates

        

In [20]:
dico_attribut_nodes,capitals, coordinates = type_point_reseau(g_reseau,nodes_livr)
nx.set_node_attributes(g_reseau, dico_attribut_nodes)

In [21]:
coordinates

{'depart': (583016.0, 6880004.5),
 'c1': (581880.5, 6880578.6),
 'c2': (581517.5, 6880556.7),
 'c3': (581517.5, 6880556.7),
 'c5': (581327.1, 6880858.1),
 'c7': (581245.2, 6881119.6),
 'c9': (581574.6, 6880937.0),
 'c10': (581767.2, 6880784.4),
 'c12': (581998.5, 6880952.4),
 'c13': (581816.7, 6881326.7),
 'c15': (582059.5, 6881069.8),
 'c17': (582137.8, 6881233.1),
 'c18': (582427.2, 6881219.0),
 'c19': (582468.8, 6880882.5),
 'c21': (582121.8, 6880761.5)}

## Calcul des distances entre chaque points

In [ ]:
short_path = nx.shortest_path(g_reseau, source=coordinates['depart'], target=coordinates['c2'])
length_path = 0.0

for p , c in enumerate(short_path[:-1]):
    edge_data = g_reseau.get_edge_data(short_path[p],short_path[p+1])
    length_path += edge_data[0]['LONGUEUR']


In [43]:
import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(liv1, liv2):
    c1 = coordinates[liv1]
    c2 = coordinates[liv2]

    short_path = nx.shortest_path(g_reseau, source=c1, target=c2)
    length_path = 0.0

    for p , c in enumerate(short_path[:-1]):
        edge_data = g_reseau.get_edge_data(short_path[p],short_path[p+1])
        length_path += edge_data[0]['LONGUEUR']



    return length_path

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}